In [1]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON, RDF, XML
from pandas import json_normalize
import pandas as pd
from rdflib import URIRef, BNode, Literal
from rdflib import Namespace
from urllib.parse import quote
import json
from datetime import datetime
import psycopg2




In [2]:
### http://UPS-SCLAB-04:7200/repositories/PracticaFinal
endPoint = "https://dbpedia.org/sparql"
sparql = SPARQLWrapper(endPoint)

In [3]:
def get_results(query):
    sparql.setReturnFormat(JSON)
    sparql.setQuery(query)
    results = sparql.query().convert()
    return(results)

In [4]:
def receive_broaders_lists(list_broaders):

    for i in list_broaders:
      
        resource = quote(i)
        query = """
        PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX dct: <http://purl.org/dc/terms/>

        select distinct ?w
            {
                {VALUES (?r) {(dbc:%s)}

                    ?subject dcterms:subject ?r.
                    ?subject foaf:isPrimaryTopicOf ?w.


                FILTER (!regex(?w, "lists"))
            }
    
        }    
        """%(resource)

    resultados = get_results(query)
    df_consulta1 = json_normalize(resultados["results"]["bindings"])
    return df_consulta1 

In [5]:
# METODO PARA OBTENER EL QUERY
def get_reource_query(resource, level):

    query = """
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dct: <http://purl.org/dc/terms/>

    select distinct ?level ?r ?c1
        {
            {VALUES (?r ?level) {(dbc:""" + resource +""" + """ + str(level) +""")}
                ?r ^skos:broader{1} ?c1.

            FILTER (!regex(?c1, "lists"))
        }
    
    }    
    """

    resultados = (get_results(query))

    df_consulta1 = json_normalize(resultados["results"]["bindings"])

    return df_consulta1



In [6]:
def obtenerRecursosWikipedia(concept, level):

    resource = quote(concept)

    query = """
        PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX dct: <http://purl.org/dc/terms/>

        select distinct ?subject as ?subject, ?w as ?wikipedia_resource, ?id_wiki as ?wikipediaId, ?level as ?nivel, ?r as ?recurso
        where
            {
                {VALUES (?r ?level) {(dbc:%s + %s)}

                    ?subject dcterms:subject ?r.
                    ?subject foaf:isPrimaryTopicOf ?w.
                    ?subject dbo:wikiPageID ?id_wiki
            }

        }    
        """ % (resource, level)

    #current_date = datetime.now()  FECHA CON SEGUNDOS Y MLS
    current_date = datetime.now().strftime("%d/%m/%Y")

    resultados = (get_results(query))

    df_conceptos_recursos = json_normalize(resultados["results"]["bindings"])

    df_conceptos_recursos['fecha_obtencion'] = current_date

    return df_conceptos_recursos

In [7]:
concept = 'Pollution'
level = 1

conceptos_recursos_df = obtenerRecursosWikipedia(concept,level)

conceptos_recursos_df.head(5)

,subject.type,subject.value,wikipedia_resource.type,wikipedia_resource.value,wikipediaId.type,wikipediaId.datatype,wikipediaId.value,nivel.type,nivel.datatype,nivel.value,recurso.type,recurso.value,fecha_obtencion
0,uri,http://dbpedia.org/resource/Ammonia_pollution,uri,http://en.wikipedia.org/wiki/Ammonia_pollution,typed-literal,http://www.w3.org/2001/XMLSchema#integer,60158522,typed-literal,http://www.w3.org/2001/XMLSchema#integer,1,uri,http://dbpedia.org/resource/Category:Pollution,23/07/2023
1,uri,http://dbpedia.org/resource/Environmental_effe...,uri,http://en.wikipedia.org/wiki/Environmental_eff...,typed-literal,http://www.w3.org/2001/XMLSchema#integer,23754906,typed-literal,http://www.w3.org/2001/XMLSchema#integer,1,uri,http://dbpedia.org/resource/Category:Pollution,23/07/2023
2,uri,http://dbpedia.org/resource/Environmental_effe...,uri,http://en.wikipedia.org/wiki/Environmental_eff...,typed-literal,http://www.w3.org/2001/XMLSchema#integer,23035042,typed-literal,http://www.w3.org/2001/XMLSchema#integer,1,uri,http://dbpedia.org/resource/Category:Pollution,23/07/2023
3,uri,http://dbpedia.org/resource/Environmental_impa...,uri,http://en.wikipedia.org/wiki/Environmental_imp...,typed-literal,http://www.w3.org/2001/XMLSchema#integer,57367229,typed-literal,http://www.w3.org/2001/XMLSchema#integer,1,uri,http://dbpedia.org/resource/Category:Pollution,23/07/2023
4,uri,http://dbpedia.org/resource/Environmental_impa...,uri,http://en.wikipedia.org/wiki/Environmental_imp...,typed-literal,http://www.w3.org/2001/XMLSchema#integer,22037708,typed-literal,http://www.w3.org/2001/XMLSchema#integer,1,uri,http://dbpedia.org/resource/Category:Pollution,23/07/2023


In [8]:
conceptos_recursos_df = conceptos_recursos_df[['recurso.value', 'nivel.value' ,'subject.value', 'wikipedia_resource.value', 'wikipediaId.value', 'fecha_obtencion']]

#conceptos_recursos_df.head()

In [9]:
unique_conceptos_df = conceptos_recursos_df.drop_duplicates(subset='subject.value')

In [10]:
#unique_conceptos_df.head()

In [11]:
#from bd import BDdatos
import codecs
import json
from datetime import date
import re 
#from bs4 import BeautifulSoup
import urllib
#import wikipedia
import requests

In [12]:
URL = "https://en.wikipedia.org/w/api.php"

S = requests.Session()


In [13]:
pageid = str(60158522)
PARAMS = {'action':'query',
        'pageids': pageid,
        'prop':'revisions|pageterms|info|pageviews',
        'rvprop':'ids|flags|timestamp|userid|user|size|comment|tags',
        'rvlimit':'500',
        'inprop':'url',
        "format": "json"
    }
req = S.get(url=URL, params=PARAMS)
data = req.json()
#date = data["query"]['pages'][pageid]['revisions'][0]['timestamp'] 

print(data)

{'batchcomplete': '', 'query': {'pages': {'60158522': {'pageid': 60158522, 'ns': 0, 'title': 'Ammonia pollution', 'revisions': [{'revid': 1104871527, 'parentid': 1104871260, 'user': 'Overtone11', 'userid': 41520451, 'timestamp': '2022-08-17T09:16:48Z', 'size': 19029, 'comment': 'Copyediting', 'tags': ['mobile edit', 'mobile web edit', 'advanced mobile edit']}, {'revid': 1104871260, 'parentid': 1099525184, 'user': 'Overtone11', 'userid': 41520451, 'timestamp': '2022-08-17T09:14:51Z', 'size': 19025, 'comment': 'Copyediting', 'tags': ['mobile edit', 'mobile web edit', 'advanced mobile edit']}, {'revid': 1099525184, 'parentid': 1095741377, 'user': 'Citation bot', 'userid': 7903804, 'timestamp': '2022-07-21T06:50:55Z', 'size': 19019, 'comment': 'Add: date, s2cid. Removed proxy/dead URL that duplicated identifier. | [[WP:UCB|Use this bot]]. [[WP:DBUG|Report bugs]]. | Suggested by Abductive | #UCB_webform 3638/3849', 'tags': []}, {'revid': 1095741377, 'parentid': 1091174043, 'user': 'GreenC b